In [2]:
!pip uninstall -y datasets gcsfs fsspec
!pip install fsspec==2024.9.0 datasets gcsfs transformers accelerate


Found existing installation: gcsfs 2024.10.0
Uninstalling gcsfs-2024.10.0:
  Successfully uninstalled gcsfs-2024.10.0
Found existing installation: fsspec 2024.10.0
Uninstalling fsspec-2024.10.0:
  Successfully uninstalled fsspec-2024.10.0
INFO: pip is looking at multiple versions of gcsfs to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00


In [3]:
import torch
import pandas as pd

In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model
model_name = "google/flan-t5-large"  # Change to flan-t5-small/large if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Data Cleaning:
- csv file of professors + ratings + tags
- csv file with baskin course offerings
- student preferences
- completed/required courses

In [5]:
### courses dicts ###

# define the courses as given
must_take = {
    "CSE 12": {"credits": 7, "prerequisites": [["CSE 5J", "CSE 20", "CSE 30", "BME 160"]]},
    "CSE 16": {"credits": 5, "prerequisites": [["MATH 20A", "MATH 19A", "MATH 19B", "MATH 11B", "AM 11B", "AM 15B", "ECON 11B"]]},
    "CSE 20": {"credits": 5, "prerequisites": []},
    "CSE 30": {"credits": 5, "prerequisites": [["CSE 20", "BME 160", "MATH 3", "MATH 11A", "MATH 19A", "AM 3", "AM 11A"]]},
    "CSE 40": {"credits": 5, "prerequisites": [["MATH 19B", "MATH 20B"], ["CSE 30"]]},
    "CSE 13S": {"credits": 7, "prerequisites": [["CSE 12", "BME 160"]]},
    "ECE 30": {"credits": 5, "prerequisites": [["MATH 19B", "MATH 20B"]]},

    "CSE 101": {"credits": 5, "prerequisites": [["CSE 12", "BME 160"], ["CSE 13E", "ECE 13", "CSE 13S"],
        ["CSE 16"], ["CSE 30"], ["MATH 11B", "MATH 19B", "MATH 20B", "AM 11B", "ECON 11B"]]},
    "CSE 101M": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 120": {"credits": 5, "prerequisites": [["CSE 12"], ["CSE 13S", "CSE 13E", "ECE 13"]]},
    "CSE 130": {"credits": 5, "prerequisites": [["CSE 12"],["CSE 101"]]}
}

# must pick 1 calc 1 class
calc_1 = {
    "MATH 19A": {"credits": 5, "prerequisites": []},
    "MATH 20A": {"credits": 5, "prerequisites": []}
}

# must pick 1 calc 2 class
calc_2 = {
    "MATH 19B": {"credits": 5, "prerequisites": [["MATH 19A", "MATH 20A"]]},
    "MATH 20B": {"credits": 5, "prerequisites": [["MATH 20A"]]}
}

# must pick 1 calc 3 class
calc_3 = {
    "AM 30": {"credits": 5, "prerequisites": [["AM 10", "MATH 21"], ["MATH 19B", "MATH 20B"]]},
    "MATH 23A": {"credits": 5, "prerequisites": [["MATH 19A"]]}
}

# must pick 1 one linear alg class
linear_alg = {
    "AM 10": {"credits": 5, "prerequisites": []},
    "MATH 21": {"credits": 5, "prerequisites": [["MATH 19A"]]}
}

# must pick 1 stats class
stats = {
    "STAT 131": {"credits": 5, "prerequisites": [["AM 11B", "ECON 11B", "MATH 11B", "MATH 19B", "MATH 20B"]]},
    "CSE 107": {"credits": 5, "prerequisites": [["CSE 16"], ["AM 30", "MATH 22", "MATH 23A"]]}
}

# must pick 1 theory class
theory = {
    "CSE 102": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 103": {"credits": 5, "prerequisites": [["CSE 101"]]}
}

# must pick 1 programming lang class
pl = {
    "CSE 112": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 114A": {"credits": 5, "prerequisites": [["CSE 101"]]}
}


# must take 3 elective fillers here, plus a capstone that counts as an elective (listed after)
electives = {
    "elective 1": {"credits": 5, "prerequisites": []},
    "elective 2": {"credits": 5, "prerequisites": []},
    "elective 3": {"credits": 5, "prerequisites": []},
}

capstones = {
    "CSE 110B": {"credits": 5, "prerequisites": [["CSE 110A"]]},
    "CSE 115C": {"credits": 5, "prerequisites": [["CSE 115B"]]},
    "CSE 115D": {"credits": 5, "prerequisites": [["CSE 115A"]]},
    "CSE 121": {"credits": 7, "prerequisites": [["CSE 12", "CSE 100", "CSE 100L"],
            ["CSE 13E", "CSE 13S", "ECE 13", "CSE 15", "CSE 15L"],
            ["ECE 101", "ECE 101L", "PHYS 5C", "PHYS 5N"]
        ]
    },
    "CSE 134": {"credits": 5, "prerequisites": [["CSE 120", "CSE 130"]]},
    "CSE 138": {"credits": 5, "prerequisites": [["CSE 130", "CSE 131"]]},
    "CSE 140": {"credits": 5, "prerequisites": [["CSE 101", ["CSE 40", "STAT 132"]]]},
    "CSE 143": {
        "credits": 5,
        "prerequisites": [
            ["CSE 101"],
            ["CSE 107", "STAT 131"],
            ["CSE 40"]
        ]
    },
    "CSE 144": {
        "credits": 5,
        "prerequisites": [
            ["CSE 40", "STAT 132"],
            ["CSE 101"]
        ]
    },
    "CSE 145": {
        "credits": 5,
        "prerequisites": [
            ["CSE 15", "CSE 15L"],
            ["CSE 30"],
            ["CSE 13S"],
            ["AM 30", "MATH 22", "MATH 23A"],
            ["STAT 5", "CSE 107", "STAT 131"],
            ["AM 10", "MATH 21"],
            ["CSE 16", "ECON 113"]
        ]
    },
    "CSE 156": {
        "credits": 5,
        "prerequisites": [["CSE 150", "CSE 101"]],
        "concurrent": "CSE 156L"
    },
    "CSE 156L": {
        "credits": 2,
        "prerequisites": [["CSE 150", "CSE 101"]],
        "concurrent": "CSE 156"
    },
    "CSE 157": {
        "credits": 7,
        "prerequisites": [["CSE 121", "CSE 150"]]
    },
    "CSE 160": {
        "credits": 7,
        "prerequisites": [["CSE 101", ["MATH 21", "AM 10"]]]
    },
    "CSE 161": {
        "credits": 5,
        "prerequisites": [["CSE 160", "equivalent"]],
        "concurrent": "CSE 161L"
    },
    "CSE 161L": {
        "credits": 2,
        "concurrent": "CSE 161"
    },
    "CSE 162": {
        "credits": 5,
        "prerequisites": [["CSE 160", "equivalent"]],
        "concurrent": "CSE 162L"
    },
    "CSE 162L": {
        "credits": 2,
        "concurrent": "CSE 162"
    },
    "CSE 163": {
        "credits": 5,
        "prerequisites": [["CSE 101"]]
    },
    "CSE 168": {
        "credits": 7,
        "prerequisites": [["CSE 160"]]
    },
    "CSE 181": {
        "credits": 5,
        "prerequisites": [["CSE 180", "CSE 130"]]
    },
    "CSE 183": {
        "credits": 5,
        "prerequisites": [
            ["CSE 15", "CSE 15L"],
            ["CMPM 35"],
            ["CSE 101"]
        ]
    },
    "CSE 184": {
        "credits": 5,
        "prerequisites": [["CSE 101"]]
    },
    "CSE 187": {
        "credits": 5,
        "prerequisites": [["CSE 186"]]
    },
    "CMPM 172": {
        "credits": 7,
        "prerequisites": [["CMPM 171"]]
    },
}

# must pick 1 DC course

DCs = {
    "CSE 115A": {"credits": 5, "prerequisites": [["CSE 101"], ["CSE 130"]]},
    "CSE 185E": {"credits": 5, "prerequisites": [["CSE 30", "CSE 12"]]},
    "CSE 185S": {"credits": 5, "prerequisites": [["CSE 101"], ["CSE 101", "CSE 101M", "CSE 120", "CSE 130"]]},
    "CSE 195": {"credits": 5, "prerequisites": []}
}

In [6]:
def flatten_prerequisites(prereq_group):
    """Flatten any nested lists within a group of prerequisites."""
    flattened = []
    for item in prereq_group:
        if isinstance(item, list):
            flattened.append(" or ".join(item))
        else:
            flattened.append(item)
    return flattened

def format_course_data(course_dict):
    formatted_courses = []
    for course, details in course_dict.items():
        # Flatten prerequisites to handle nested lists
        prereqs = ", ".join(
            [" or ".join(flatten_prerequisites(group)) for group in details.get("prerequisites", [])]
        ) or "none"
        formatted_courses.append(f"{course} ({details['credits']} credits, prerequisites: {prereqs})")
    return "\n".join(formatted_courses)

# Example: Format must_take courses
must_take_formatted = format_course_data(must_take)
calc_1_formatted = format_course_data(calc_1)
calc_2_formatted = format_course_data(calc_2)
calc_3_formatted = format_course_data(calc_3)
linear_alg_formatted = format_course_data(linear_alg)
stats_formatted = format_course_data(stats)
theory_formatted = format_course_data(theory)
pl_formatted = format_course_data(pl)
electives_formatted = format_course_data(electives)
capstones_formatted = format_course_data(capstones)
DCs_formatted = format_course_data(DCs)

# print(must_take_formatted)
# print(calc_1_formatted)
# print(calc_2_formatted)

Create sample students with 5 tags each

In [7]:
import random

# 0 if bad, 1 if good
tags = {"EXTRA CREDIT":1, "Amazing lectures":1, "Clear grading criteria":1, "Gives good feedback":1, "Inspirational":1, "Hilarious":1, "Caring":1, "Respected":1, "Accessible outside class":1, "Online savvy":1, "Tough grader":0, "Get ready to read":0, "Lots of homework":0, "Beware of pop quizzes":0, "So many papers":0, "Lecture heavy":0, "Test heavy":0, "Graded by few things":0, "Participation matters":0, "Group projects":0}
# List of tag names
tag_names = list(tags.keys())

# Generate 100 sample students
sample_students = [random.sample(tag_names, 5) for _ in range(100)]

# Print a few examples
print(sample_students[:5])

[['Participation matters', 'Beware of pop quizzes', 'Graded by few things', 'Online savvy', 'Lots of homework'], ['Caring', 'Test heavy', 'Clear grading criteria', 'EXTRA CREDIT', 'Respected'], ['Test heavy', 'Caring', 'Lecture heavy', 'Online savvy', 'EXTRA CREDIT'], ['So many papers', 'Accessible outside class', 'Lecture heavy', 'Test heavy', 'Caring'], ['Lecture heavy', 'Hilarious', 'Inspirational', 'Amazing lectures', 'Tough grader']]


In [ ]:
import pandas as pd
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Get the file name (assuming only one file is uploaded)
file_name = list(uploaded.keys())[0]

# Read the CSV into a pandas DataFrame
df = pd.read_csv(file_name)

# Display the DataFrame
print(df.head())

Saving professors_with_ids_and_tags.csv to professors_with_ids_and_tags (1).csv
            department   quarter class_name            professor_name  \
0  Applied Mathematics    Fall24        AM3          Jelliffe Jackson   
1  Applied Mathematics    Fall24       AM10        Yonatan Katznelson   
2  Applied Mathematics  Winter25       AM10  Jonathan Rice Katznelson   
3  Applied Mathematics  Spring25       AM10  Jonathan Rice Katznelson   
4  Applied Mathematics    Fall24      AM11A              Julie Simons   

      section  professor_rating  professor_difficulty  professor_num_ratings  \
0  Section 01               4.4                   2.1                     62   
1  Section 01               2.8                   3.8                    501   
2  Section 01               3.2                   3.8                     44   
3  Section 01               3.2                   3.8                     44   
4  Section 01               4.3                   3.1                     85   



In [ ]:
# As bullet points
text_bullets = "\n".join(
    [f"- {', '.join(f'{col}: {val}' for col, val in row.items())}" for _, row in df.iterrows()]
)
print(text_bullets)

- department: Applied Mathematics, quarter: Fall24, class_name: AM3, professor_name: Jelliffe Jackson, section: Section 01, professor_rating: 4.4, professor_difficulty: 2.1, professor_num_ratings: 62, professor_id: 2874413.0, tags: ['Clear grading criteria', 'Test heavy', 'Online Savvy', 'EXTRA CREDIT', 'Group projects']
- department: Applied Mathematics, quarter: Fall24, class_name: AM10, professor_name: Yonatan Katznelson, section: Section 01, professor_rating: 2.8, professor_difficulty: 3.8, professor_num_ratings: 501, professor_id: 2303587.0, tags: ['Test heavy', 'Lecture heavy', 'Clear grading criteria', 'Tough grader', 'Beware of pop quizzes']
- department: Applied Mathematics, quarter: Winter25, class_name: AM10, professor_name: Jonathan Rice Katznelson, section: Section 01, professor_rating: 3.2, professor_difficulty: 3.8, professor_num_ratings: 44, professor_id: 464548.0, tags: []
- department: Applied Mathematics, quarter: Spring25, class_name: AM10, professor_name: Jonathan 

In [ ]:
# Combine all the class dictionaries into a single set of class names
all_prereq_classes = set(
    cls.replace(" ", "")  # Remove all spaces from each class name
    for cls in (list(must_take.keys()) + list(calc_1.keys()) + list(calc_2.keys()) +
                list(calc_3.keys()) + list(linear_alg.keys()) + list(stats.keys()) +
                list(theory.keys()) + list(pl.keys()) + list(DCs.keys()) + list(electives.keys()) + list(capstones.keys()))
)
#  + list(electives.keys()) + list(capstones.keys())

# print(all_prereq_classes)
# Filter the DataFrame to include only classes in the prerequisites
filtered_df = df[df['class_name'].isin(all_prereq_classes)]
# print(df.head())
# print(filtered_df.head())

# Generate formatted text from the filtered DataFrame
filtered_class_data = "\n".join(
    f"Class: {row['class_name']}, Professor: {row['professor_name']}, Rating: {row['professor_rating']}, "
    f"Tags: {row['tags']}, Quarter: {row['quarter']}"
    for _, row in filtered_df.iterrows()
)

print(filtered_class_data)
print(len(filtered_class_data))

Class: AM10, Professor: Yonatan Katznelson, Rating: 2.8, Tags: ['Test heavy', 'Lecture heavy', 'Clear grading criteria', 'Tough grader', 'Beware of pop quizzes'], Quarter: Fall24
Class: AM10, Professor: Jonathan Rice Katznelson, Rating: 3.2, Tags: [], Quarter: Winter25
Class: AM10, Professor: Jonathan Rice Katznelson, Rating: 3.2, Tags: [], Quarter: Spring25
Class: AM30, Professor: Javier Gonzalez-Rocha, Rating: 2.4, Tags: ['Caring', 'Accessible outside class', 'Hilarious', 'Participation matters', 'Gives good feedback'], Quarter: Fall24
Class: CMPM172, Professor: Tyler Coleman, Rating: 4.8, Tags: ['Caring', 'Clear grading criteria', 'Respected', 'Inspirational', 'Amazing lectures '], Quarter: Spring25
Class: CMPM172, Professor: Tyler Coleman, Rating: 4.8, Tags: ['Caring', 'Clear grading criteria', 'Respected', 'Inspirational', 'Amazing lectures '], Quarter: Spring25
Class: CSE12, Professor: Sagnik Nath, Rating: 2.5, Tags: ['Lecture heavy', 'Tough grader', 'EXTRA CREDIT', 'Hilarious', 

course info:
- course_name
- quarters_offered
- professor
- prereqs*

prof info:
- prof name
- rating
- difficulty
- tags x3*

student info:
- classes to take
- classes taken
- preferences / tags

CS info:
- required classes


Prompt Engineering

create schedules using prompt engineering (based off of several examples)



for each class, check which prof is teaching that quarter and get rating

create score to match with that schedule.

In [33]:
import re

# test prompting

# base prompt
input_text = """"""

# USE THIS VERSION
input_text_2_shot = """Create a sample 4 year schedule for a Computer Science major at UCSC
It should contain which classes to take in each of the 12 total quarters.
Ensure all major requirements are met and take prerequesites into account.
Here are two example schedules which include all the required classes (including 3 elective classes and 1 capstone class)

Example Schedule 1:
1	['MATH 19A', 'CSE 20']
2	['MATH 19B', 'AM 10']
3	['MATH 23A', 'CSE 30']
4	['CSE 16', 'CSE 12']
5	['CSE 13S', 'ECE 30']
6	['CSE 40', 'CSE 101']
7	['CSE 107', 'CSE 120']
8	['CSE 130', 'CSE 101M']
9	['CSE 103', 'CSE 195']
10	['ELECTIVE', 'CSE 114A']
11	['ELECTIVE']
12	['ELECTIVE', 'CAPSTONE']

Example Schedule 2:
1	['MATH 19A', 'CSE 20']
2	['MATH 19B', 'CSE 30']
3	['MATH 23A', 'CSE 195']
4	['ECE 30', 'CSE 40']
5	['MATH 21', 'CSE 16']
6	['CSE 12', 'STAT 131']
7	['CSE 13S']
8	['CSE 120', 'CSE 101']
9	['CSE 130', 'CSE 101M']
10	['ELECTIVE', 'CSE 102']
11	['ELECTIVE', 'CSE 112']
12	['ELECTIVE', 'CAPSTONE']

Generate a new schedule with exactly 12 quarters with all the required classes.
New Schedule:
"""

# reaches token limit
input_text_3_shot = """
Create a sample 4 year schedule for a Computer Science major at UCSC
It should contain which classes to take in each of the 12 total quarters.
Ensure all major requirements are met and take prerequesites into account.
Here are three example schedules which include all the required classes (including 3 elective classes and 1 capstone class)
Example Schedule 1:
1	['MATH 19A', 'CSE 20']
2	['MATH 19B', 'AM 10']
3	['MATH 23A', 'CSE 30']
4	['CSE 16', 'CSE 12']
5	['CSE 13S', 'ECE 30']
6	['CSE 40', 'CSE 101']
7	['CSE 107', 'CSE 120']
8	['CSE 130', 'CSE 101M']
9	['CSE 103', 'CSE 195']
10	['ELECTIVE', 'CSE 114A']
11	['ELECTIVE']
12	['ELECTIVE', 'CAPSTONE']

Example Schedule 2:
1	['MATH 19A', 'CSE 20']
2	['MATH 19B', 'CSE 30']
3	['MATH 23A', 'CSE 195']
4	['ECE 30', 'CSE 40']
5	['MATH 21', 'CSE 16']
6	['CSE 12', 'STAT 131']
7	['CSE 13S']
8	['CSE 120', 'CSE 101']
9	['CSE 130', 'CSE 101M']
10	['ELECTIVE', 'CSE 102']
11	['ELECTIVE', 'CSE 112']
12	['ELECTIVE', 'CAPSTONE']

Example Schedule 3:
1	['MATH 20A', 'AM 10']
2	['MATH 20B', 'CSE 20']
3	['MATH 23A', 'CSE 16']
4	['CSE 107', 'ECE 30']
5	['CSE 30', 'CSE 12']
6	['CSE 13S', 'CSE 40']
7	['CSE 120', 'CSE 101']
8	['CSE 101M', 'CSE 130']
9	['CSE 102', 'CSE 114A']
10	['ELECTIVE', 'CSE 115A']
11	['ELECTIVE']
12	['ELECTIVE', 'CAPSTONE']

"""

# reaches token limit
input_text_cot = """
Create a 4-year schedule for a Computer Science major at UCSC with 3 quarters per year (12 quarters total).
Follow these steps:

1. List all major requirements and prerequisites.
2. Assign classes to each quarter, starting with prerequisites and ensuring they are met before higher-level classes.
3. Include electives in the later quarters to meet the requirement of 12 total quarters.

Examples:

Schedule 1:
Step 1: Major requirements and prerequisites:
- Prerequisites: MATH 19A -> MATH 19B -> MATH 23A
- Core classes: CSE 20, CSE 30, CSE 16, CSE 12, CSE 13S, etc.
- Advanced electives: CSE 101, CSE 107, CSE 120, CSE 130, etc.
- Capstone: CAPSTONE
Step 2: Assign classes to quarters:
1 ['MATH 19A', 'CSE 20']
2 ['MATH 19B', 'AM 10']
3 ['MATH 23A', 'CSE 30']
...
12 ['ELECTIVE', 'CAPSTONE']

Schedule 2:
Step 1: Major requirements and prerequisites:
- Prerequisites: MATH 19A -> MATH 19B -> MATH 23A
- Core classes: CSE 20, CSE 30, CSE 16, CSE 12, CSE 13S, etc.
- Advanced electives: CSE 101, CSE 107, CSE 120, CSE 130, etc.
- Capstone: CAPSTONE
Step 2: Assign classes to quarters:
1 ['MATH 19A', 'CSE 20']
2 ['MATH 19B', 'CSE 30']
3 ['MATH 23A', 'CSE 195']
...
12 ['ELECTIVE', 'CAPSTONE']

Now, create a new schedule using this reasoning format:
Step 1: List the major requirements and prerequisites.
Step 2: Assign classes to each of the 12 quarters:
"""

inputs = tokenizer(input_text_2_shot, return_tensors="pt") # change input text

# Generate output
outputs = model.generate(**inputs, max_length = 1024)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the result
print("Input:", input_text_2_shot) # change input text name
print("Generated Output:", output_text)

# Split the output into individual schedules
cleaned_output = re.sub(r"(?<=\])\s*\d+\s*(?=\[)", " ", output_text)

schedules = cleaned_output.split("] [")
schedules = [s.strip("[]") for s in schedules]

# Create a DataFrame with quarter numbers
df = pd.DataFrame({
    "Quarter": range(1, len(schedules) + 1),
    "Classes": [f"[{schedule}]" for schedule in schedules]
})

# Save to CSV
csv_filename = "schedule_2_shot_v2.csv" # change filename
df.to_csv(csv_filename, index=False)

print(f"Schedule saved to {csv_filename}")

with open(csv_filename, "r") as file:
    print(file.read())

# save file to drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
new_csv_filename = '/content/drive/My Drive/' + csv_filename
df.to_csv(new_csv_filename, index=False)

Input: Create a sample 4 year schedule for a Computer Science major at UCSC 
It should contain which classes to take in each of the 12 total quarters. 
Ensure all major requirements are met and take prerequesites into account.   
Here are two example schedules which include all the required classes (including 3 elective classes and 1 capstone class) 

Example Schedule 1: 
1	['MATH 19A', 'CSE 20']
2	['MATH 19B', 'AM 10']
3	['MATH 23A', 'CSE 30']
4	['CSE 16', 'CSE 12']
5	['CSE 13S', 'ECE 30']
6	['CSE 40', 'CSE 101']
7	['CSE 107', 'CSE 120']
8	['CSE 130', 'CSE 101M']
9	['CSE 103', 'CSE 195']
10	['ELECTIVE', 'CSE 114A']
11	['ELECTIVE']
12	['ELECTIVE', 'CAPSTONE']

Example Schedule 2: 
1	['MATH 19A', 'CSE 20']
2	['MATH 19B', 'CSE 30']
3	['MATH 23A', 'CSE 195']
4	['ECE 30', 'CSE 40']
5	['MATH 21', 'CSE 16']
6	['CSE 12', 'STAT 131']
7	['CSE 13S']
8	['CSE 120', 'CSE 101']
9	['CSE 130', 'CSE 101M']
10	['ELECTIVE', 'CSE 102']
11	['ELECTIVE', 'CSE 112']
12	['ELECTIVE', 'CAPSTONE']

Generate a new

In [35]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Output CSV filename (saved in Drive)
csv_filename = '/content/drive/My Drive/schedule_2_shot_many.csv'

# Initialize the CSV file by writing headers (only for the first run)
with open(csv_filename, "w") as f:
    f.write("Quarter,Classes\n")

# Loop to generate and append 10 schedules
for i in range(3):
    print(f"Generating schedule {i + 1}...")

    # Generate output
    inputs = tokenizer(input_text_2_shot, return_tensors="pt")
    # outputs = model.generate(**inputs, max_length=1024)
    outputs = model.generate(
        **inputs,
        max_length=1024,
        temperature=1.0,  # High temperature for more randomness
        top_p=0.9,  # Include a wider range of tokens
        top_k=100,  # Allow for more token options
        do_sample=True  # Enable sampling instead of greedy decoding
    )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean and parse the output
    cleaned_output = re.sub(r"(?<=\])\s*\d+\s*(?=\[)", " ", output_text)
    schedules = cleaned_output.split("] [")
    schedules = [s.strip("[]") for s in schedules]

    # Create a DataFrame with quarter numbers
    df = pd.DataFrame({
        "Quarter": range(1, len(schedules) + 1),
        "Classes": [f"[{schedule}]" for schedule in schedules]
    })

    # Append to the existing CSV file
    df.to_csv(csv_filename, mode="a", header=False, index=False)

print(f"All schedules saved to {csv_filename}")

# Read and display the final CSV content
with open(csv_filename, "r") as file:
    print(file.read())

Mounted at /content/drive
Generating schedule 1...
Generating schedule 2...
Generating schedule 3...
All schedules saved to /content/drive/My Drive/schedule_2_shot_many.csv
Quarter,Classes
1,"['RELAX', 'MATH', 'CSE', 'ENGINEERING', 'BUSINESS MANAGEMENT', 'FOLLOWING RESULTS', 'ELECTIVE', 'BASICS', 'JOURNALISM', 'ENGINEERING', 'COMPUTER STUDENTS', 'JUNIOR COLLEGE', 'ELECTIVE', 'HIGHEST', 'FORMAT ', 'COOL', 'VOLUNTEERS', 'DIFFERENTIAL ENGINEERING', 'CONCULTURAL STUDENTS', 'FOLLOWING RESULTS', 'INTERNATIONAL STUDENTS', 'PRACTICAL STUDENTS', 'BUSINESS', 'EXPERIMENTAL ENGINEERING', 'BUSINESS', 'EMBASSY ', 'DEFINITIVE', 'COMPUTER ENGINEERING', 'ENGINEERING']"
1,"['MATH 19A, 'CSE 20']"
2,"['MATH 19B, 'CSE 30']"
3,"['MATH 23A, 'CSE 30']"
4,"['CSE 16', 'CSE 12']"
5,"['CSE 13S, 'ECE 30']"
6,"['CSE 40', 'CSE 101']"
7,"['CSE 107, 'CSE 120']"
8,"['CSE 130, 'CSE 101M']"
9,"['CSE 103, 'CSE 195']"
10,['ELECTIVE']
11,['ELECTIVE']
1,"['MATH 19A', 'CSE 20']"
2,"['MATH 19B', 'AM 10']"
3,"['MATH 23A', 'CSE 

Prompting with trying to include requirements / other information about classes in the prompt

In [ ]:
# Define the prompt
prompt = """
Generate a sample 4-year schedule for a Computer Science major at UCSC.
Use the classes listed below and ensure all major requirements are met. Take into account classes that have prerequesites.
Replace placeholders (e.g., [Class1, Class2]) with real class names from the provided data.
"""
prompt2 = """
Provide the output in the following format:
Fall24: [Class1, Class2]
Winter25: [Class3, Class4]
Spring25 [Class5, Class6]
etc. """

final_prompt = prompt + "\nRequired classes and electives:\n" + str(all_prereq_classes) + prompt2 + "\nSchedule:"
# + "\nStudent Data:\n" + str(sample_students[0])
# "\nClass Data:\n" + filtered_class_data +
print(final_prompt)

# Tokenize input
inputs = tokenizer(final_prompt, return_tensors="pt", max_length=1024, truncation=True).to(device)

# Generate output
outputs = model.generate(inputs.input_ids, max_length=150, num_beams=5, early_stopping=True)

# Decode and print the output
schedule = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Schedule:\n", schedule)



Generate a sample 4-year schedule for a Computer Science major at UCSC. 
Use the classes listed below and ensure all major requirements are met. Take into account classes that have prerequesites. 
Replace placeholders (e.g., [Class1, Class2]) with real class names from the provided data.

Required classes and electives:
{'MATH23A', 'CSE115C', 'CSE183', 'CSE121', 'CSE161', 'CSE115A', 'CSE20', 'CSE101M', 'MATH21', 'CSE162L', 'CMPM172', 'CSE138', 'CSE140', 'CSE115D', 'elective1', 'elective3', 'CSE13S', 'CSE120', 'CSE161L', 'CSE134', 'elective2', 'CSE160', 'CSE107', 'MATH20B', 'CSE101', 'CSE185E', 'MATH20A', 'CSE156', 'CSE30', 'AM30', 'STAT131', 'CSE102', 'CSE145', 'CSE181', 'CSE184', 'CSE114A', 'CSE110B', 'CSE103', 'CSE16', 'CSE143', 'AM10', 'CSE168', 'CSE40', 'CSE157', 'CSE162', 'MATH19B', 'CSE185S', 'CSE195', 'MATH19A', 'CSE156L', 'CSE187', 'ECE30', 'CSE130', 'CSE144', 'CSE12', 'CSE163', 'CSE112'}
Provide the output in the following format:
Fall24: [Class1, Class2]
Winter25: [Class3, C

Fine tuning model (if we get to this point)

In [ ]:
from datasets import load_dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load dataset
dataset = load_dataset("csv", data_files={"train": "train.csv", "validation": "val.csv"})

# Tokenize the dataset
def preprocess_function(examples):
    inputs = tokenizer(examples["Input"], max_length=512, truncation=True)
    labels = tokenizer(examples["Output"], max_length=512, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan_t5_finetuned",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    predict_with_generate=True,
    fp16=True  # Use mixed precision for faster training on GPUs
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer
)

# Train the model
trainer.train()


Save finetuned model

In [ ]:
model.save_pretrained("./flan_t5_finetuned")
tokenizer.save_pretrained("./flan_t5_finetuned")